## Read Nirvana JSON output by lines

In [1]:
file = gzfile('S1P_dragen_subset.json.gz', 'rt')

header = ''
positions = c()
genes = c()
is_header_line = TRUE
is_position_line = FALSE
is_gene_line = FALSE
gene_section_line = '],"genes":['
end_line = ']}'
position_count = 0
gene_count = 0
while ( TRUE ) {
    line = readLines(file, n = 1)
    trim_line = trimws(line)
    if ( is_header_line ) {
        ## only keep the "header" field content from the line
        header = substr(trim_line, 11, nchar(trim_line)-14)
        is_header_line = FALSE
        is_position_line = TRUE
        next
    }
    if ( trim_line == gene_section_line ) {
        is_gene_line = TRUE
        is_position_line = FALSE
        next
    }
    else if ( trim_line == end_line ) {
        break
    }
    else {
        if ( is_position_line ) {
            ## remove the trailing ',' if there is
            positions = c(positions, sub(",$", "", trim_line))
            position_count = position_count + 1
        }
        if ( is_gene_line ) {
            ## remove the trailing ',' if there is
            genes = c(genes, sub(",$", "", trim_line))
            gene_count = gene_count + 1
        }
    }
}
close(file)
print(paste('header object: ', header))
print(paste('number of positions: ', position_count))
print(paste('number of genes: ', gene_count))

[1] "header object:  {\"annotator\":\"Nirvana 3.16.1\",\"creationTime\":\"2022-05-21 10:27:54\",\"genomeAssembly\":\"GRCh38\",\"schemaVersion\":6,\"dataVersion\":\"91.27.63\",\"dataSources\":[{\"name\":\"VEP\",\"version\":\"91\",\"description\":\"BothRefSeqAndEnsembl\",\"releaseDate\":\"2017-12-18\"},{\"name\":\"MultiZ100Way\",\"version\":\"20171006\",\"description\":\"Amino acid conservation scores calculated from MultiZ100Way alignments from UCSC.\",\"releaseDate\":\"2017-10-06\"},{\"name\":\"ClinVar\",\"version\":\"20211202\",\"description\":\"A freely accessible, public archive of reports of the relationships among human variations and phenotypes, with supporting evidence\",\"releaseDate\":\"2021-12-02\"},{\"name\":\"dbSNP\",\"version\":\"154\",\"description\":\"Identifiers for observed variants\",\"releaseDate\":\"2020-05-01\"},{\"name\":\"dbSNP\",\"version\":\"151\",\"description\":\"Identifiers for observed variants\",\"releaseDate\":\"2018-04-18\"},{\"name\":\"gnomAD\",\"versio

## Retrieve variants under a gnomAD allele frequency threshold

In [2]:
library(jsonlite)

freq_threshold = 0.0001
ids = c()
freqs = c()

for (position in positions) {
    position_data = fromJSON(position)
    for (i in 1:dim(position_data$variants)[1]) {
        vid = position_data$variants$vid[i]
        freq = position_data$variants$gnomad$allAf[i]
        if ( !is.null(freq) && !is.na(freq) && freq < freq_threshold ) {
            ids = c(ids, vid)
            freqs = c(freqs, freq)
        }
    }
}

freq_df = cbind(ids, freqs)
colnames(freq_df) = c("variant_id", "gnomAD_allele_freq")
print(dim(freq_df))
freq_df[1:20,]

[1] 90  2


variant_id,gnomAD_allele_freq
1-28198-A-T,0
1-46402-C-CT,0
1-55119-C-G,2.8e-05
1-59343-C-T,2.3e-05
1-62794-T-A,2.3e-05
1-64927-G-T,8e-06
1-66269-A-T,0
1-66356-T-A,8.8e-05
1-70991-C-T,0
1-72456-G-A,0


## Retrieve all relevant genes and their OMIM gene names

In [5]:
gene_symbols = list()
omim_gene_names = list()

for (gene in genes) {
    gene_data = fromJSON(gene)
    gene_symbols = append(gene_symbols, gene_data$name)
    omim_gene_name = gene_data$omim$geneName[1]
    if ( is.null(omim_gene_name) ) {
        omim_gene_names = append(omim_gene_names, list(NULL))
    }
    else {
        omim_gene_names = append(omim_gene_names, omim_gene_name)
    }
}

gene_df = cbind(gene_symbols, omim_gene_names)
colnames(gene_df) = c("gene", "OMIM_gene_name")
gene_df

gene       OMIM_gene_name                                               
 [1,] ACAP3      NULL                                                         
 [2,] AGRN       Agrin                                                        
 [3,] AL627309.1 NULL                                                         
 [4,] AL645608.1 NULL                                                         
 [5,] B3GALT6    UDP-Gal:beta-Gal beta-1,3-galactosyltransferase polypeptide 6
 [6,] C1QTNF12   Family with sequence similarity 132, member A                
 [7,] C1orf159   NULL                                                         
 [8,] CPTP       Glycolipid transfer protein domain-containing protein 1      
 [9,] DVL1       Dishevelled segment polarity protein 1                       
[10,] HES4       Hes family bHLH transcription factor 4                       
[11,] INTS11     Cleavage and polyadenylation-specific factor 3-like          
[12,] ISG15      ISG15 ubiquitin-like modifier                                
[13,] KLHL17     Kelch-like 17                                                
[14,] NOC2L      NOC2-like nucleolar-associated transcriptional repressor     
[15,] OR4F5      NULL                                                         
[16,] PERM1      PPARGC1-and ESRR-induced regulator, muscle, 1                
[17,] PLEKHN1    NULL                                                         
[18,] PUSL1      NULL                                                         
[19,] RNF223     NULL                                                         
[20,] SAMD11     Sterile alpha motif domain-containing protein 11             
[21,] SCNN1D     Sodium channel epithelial 1, subunit delta                   
[22,] SDF4       Stromal cell-derived factor 4                                
[23,] TAS1R3     Taste receptor type 1, member 3                              
[24,] TNFRSF18   Tumor necrosis factor receptor superfamily, member 18        
[25,] TNFRSF4    Tumor necrosis factor receptor superfamily, member 4         
[26,] TTLL10     NULL                                                         
[27,] UBE2J2     NULL